In [2]:
import numpy as np
from netCDF4 import Dataset
import glob
import math
from scipy.interpolate import interp1d

# Function to rotate wind components
# def rotate_winds(u, v, rot_grid):
#     u_rot = np.zeros_like(u)
#     v_rot = np.zeros_like(v)
#     for z in range(u.shape[0]):
#         v_rot[z, :, :] = np.cos(rot_grid) * v[z, :, :] - np.sin(rot_grid) * u[z, :, :]
#         u_rot[z, :, :] = np.sin(rot_grid) * v[z, :, :] + np.cos(rot_grid) * u[z, :, :]
#     return u_rot, v_rot

# Function to interpolate 3D data to a given lat/lon point
def interpolate_to_point(lat_grid, lon_grid, data_3d, lat_point, lon_point):
    from scipy.interpolate import griddata
    points = np.column_stack((lat_grid.ravel(), lon_grid.ravel()))
    data_interp = np.zeros((data_3d.shape[0],))
    for z in range(data_3d.shape[0]):
        values = data_3d[z, :, :].ravel()
        data_interp[z] = griddata(points, values, (lat_point, lon_point), method='linear')
    return data_interp


In [5]:
# Latitude and longitude point to interpolate to
LATpoint = 39.969
LONpoint = -72.717

iz = np.arange(0, 5, 1)  # Range of indices to extract from the 3D data
zo = np.linspace(20.0, 200.0, 10)  # Altitudes to interpolate to

# List of HRRR analysis files
flist = sorted(glob.glob('/Volumes/hrrr/full/*.nc'))

In [ ]:

for n, file in enumerate(flist):
    with Dataset(file, 'r') as nc:
        Z3D = nc.variables['HGT'][iz, :, :]
        U3Do = nc.variables['UGRD'][iz, :, :]
        V3Do = nc.variables['VGRD'][iz, :, :]

        # Rotate winds
        #U3D, V3D = rotate_winds(U3Do, V3Do, ROTgrid)

        # Interpolate to the desired lat/lon point
        Upoint[n, :] = interpolate_to_point(LATgrid, LONgrid, U3Do, LATpoint, LONpoint)
        Vpoint[n, :] = interpolate_to_point(LATgrid, LONgrid, V3Do, LATpoint, LONpoint)
        Zpoint[n, :] = interpolate_to_point(LATgrid, LONgrid, Z3D, LATpoint, LONpoint)
        print(n)

In [ ]:
# Vertical interpolation
Upoint_z = np.array([interp1d(Zpoint[n, :], Upoint[n, :], kind='linear')(zo) for n in range(Nt)])
Vpoint_z = np.array([interp1d(Zpoint[n, :], Vpoint[n, :], kind='linear')(zo) for n in range(Nt)])

In [ ]:

# Save results to a new NetCDF file
with Dataset("HRRR_point_interp2Z.nc", "w", format="NETCDF4") as fout:
    fout.createDimension('time', Nt)
    fout.createDimension('altitude', len(zo))

    times = fout.createVariable('time', 'f4', ('time',))
    altitudes = fout.createVariable('altitude', 'f4', ('altitude',))

    times[:] = np.arange(Nt)
    altitudes[:] = zo

    u_point = fout.createVariable('Upoint', 'f4', ('time', 'altitude'))
    v_point = fout.createVariable('Vpoint', 'f4', ('time', 'altitude'))
    z_point = fout.createVariable('Zpoint', 'f4', ('time', 'altitude'))
    u_point_z = fout.createVariable('Upoint_z', 'f4', ('time', 'altitude'))
    v_point_z = fout.createVariable('Vpoint_z', 'f4', ('time', 'altitude'))

    u_point[:] = Upoint
    v_point[:] = Vpoint
    z_point[:] = Zpoint
    u_point_z[:] = Upoint_z
    v_point_z[:] = Vpoint_z

print("done")